# Self-Contained ATC Training Environment

**Training AI for Air Traffic Control - No External Dependencies**

This notebook is **completely self-contained** with its own visual simulation:

## Features

1. **Visual Canvas**: Black background with intersecting white runways
2. **Aircraft Simulation**: Yellow triangles showing heading/position
3. **Realistic Physics**: Based on OpenScope dynamics
4. **Safety Rules**: 3nm lateral, 1000ft vertical separation
5. **RL Training**: Full Gymnasium + Stable-Baselines3 integration

**No Browser Required** - Pure Python simulation!

---

# Section 1: Imports and Configuration

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Polygon
from typing import Dict, List, Tuple, Optional, Any
import gymnasium as gym
from gymnasium import spaces
from dataclasses import dataclass
import math
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from IPython.display import clear_output
import time

print('✅ Imports loaded')

✅ Imports loaded


# Section 2: Aircraft and Physics Simulation

In [4]:
# Constants (based on OpenScope)
SEPARATION_LATERAL_NM = 3.0  # 3 nautical miles
SEPARATION_VERTICAL_FT = 1000.0  # 1000 feet
CONFLICT_WARNING_BUFFER_NM = 1.0  # Additional warning buffer
NM_TO_KM = 1.852  # Nautical miles to kilometers
KM_TO_NM = 1.0 / NM_TO_KM
TURN_RATE_DEG_PER_SEC = 3.0  # 3 degrees per second
FT_PER_SEC_CLIMB = 2000 / 60  # ~2000 fpm = 33.3 ft/s
FT_PER_SEC_DESCENT = 2000 / 60

# Simulation area (10x10 nm around airport)
AREA_SIZE_NM = 20.0

# Aircraft call signs
CALLSIGNS = [
    "AAL123", "UAL456", "DAL789", "SWA101", "FDX202",
    "JBU303", "ASA404", "SKW505", "NKS606", "FFT707",
    "AAL808", "UAL909", "DAL111", "SWA222", "FDX333",
    "JBU444", "ASA555", "SKW666", "NKS777", "FFT888"
]

print('✅ Constants defined')

✅ Constants defined


In [ ]:
@dataclass
class Aircraft:
    """Aircraft with realistic dynamics."""
    callsign: str
    x: float  # Position in nm (relative to airport)
    y: float  # Position in nm (relative to airport)
    altitude: float  # Altitude in feet
    heading: float  # Heading in degrees (0-360, 0=North)
    speed: float  # Speed in knots
    
    # Commanded values
    target_altitude: float = None
    target_heading: float = None
    target_speed: float = None
    
    # State
    is_landing: bool = False
    runway_assigned: int = None  # 0, 1, 2, 3 for four runways
    
    def __post_init__(self):
        if self.target_altitude is None:
            self.target_altitude = self.altitude
        if self.target_heading is None:
            self.target_heading = self.heading
        if self.target_speed is None:
            self.target_speed = self.speed
    
    def update(self, dt: float):
        """Update aircraft state based on physics (dt in seconds)."""
        # Update heading (turn at TURN_RATE_DEG_PER_SEC)
        heading_diff = self.target_heading - self.heading
        # Normalize to [-180, 180]
        heading_diff = (heading_diff + 180) % 360 - 180
        
        max_turn = TURN_RATE_DEG_PER_SEC * dt
        if abs(heading_diff) <= max_turn:
            self.heading = self.target_heading
        else:
            self.heading += max_turn * np.sign(heading_diff)
        self.heading = self.heading % 360
        
        # Update altitude
        alt_diff = self.target_altitude - self.altitude
        if abs(alt_diff) > 0:
            climb_rate = FT_PER_SEC_CLIMB if alt_diff > 0 else -FT_PER_SEC_DESCENT
            max_alt_change = abs(climb_rate * dt)
            if abs(alt_diff) <= max_alt_change:
                self.altitude = self.target_altitude
            else:
                self.altitude += max_alt_change * np.sign(alt_diff)
        
        # Update speed (instant for simplicity)
        self.speed = self.target_speed
        
        # Update position based on heading and speed
        # Convert heading to radians (0 deg = North = +Y)
        heading_rad = np.radians(self.heading)
        
        # Speed in nm/s
        speed_nm_per_sec = self.speed / 3600.0
        
        # Update x, y (heading 0 = North = +y, heading 90 = East = +x)
        self.x += speed_nm_per_sec * dt * np.sin(heading_rad)
        self.y += speed_nm_per_sec * dt * np.cos(heading_rad)
    
    def distance_to(self, other: 'Aircraft') -> float:
        """Calculate lateral distance in nautical miles."""
        return np.sqrt((self.x - other.x)**2 + (self.y - other.y)**2)
    
    def vertical_separation(self, other: 'Aircraft') -> float:
        """Calculate vertical separation in feet."""
        return abs(self.altitude - other.altitude)
    
    def check_conflict(self, other: 'Aircraft') -> Tuple[bool, bool]:
        """Check for conflicts and violations.
        
        Returns:
            (is_violation, is_conflict)
        """
        lateral_dist = self.distance_to(other)
        vertical_sep = self.vertical_separation(other)
        
        # Check vertical separation
        if vertical_sep >= SEPARATION_VERTICAL_FT:
            return False, False
        
        # Check lateral separation
        is_violation = lateral_dist < SEPARATION_LATERAL_NM
        is_conflict = lateral_dist < (SEPARATION_LATERAL_NM + CONFLICT_WARNING_BUFFER_NM)
        
        return is_violation, is_conflict

print('✅ Aircraft class defined')

✅ Aircraft class defined


# Section 3: Runway Environment with Visualization

In [1]:
class RunwayEnvironment(gym.Env):
    """Self-contained ATC environment with visual rendering."""
    
    metadata = {'render_modes': ['human', 'rgb_array'], 'render_fps': 4}
    
    def __init__(
        self,
        max_aircraft: int = 10,
        episode_length: int = 300,  # seconds
        spawn_interval: float = 30.0,  # spawn every 30 seconds
        render_mode: Optional[str] = None,
    ):
        super().__init__()
        
        self.max_aircraft = max_aircraft
        self.episode_length = episode_length
        self.spawn_interval = spawn_interval
        self.render_mode = render_mode
        
        # Runway configuration (two intersecting runways)
        # Runway 1: North-South (09/27 - East/West in aviation terms)
        # Runway 2: Northeast-Southwest (04/22)
        self.runways = [
            {'name': '09', 'heading': 90, 'x1': -2, 'y1': 0, 'x2': 2, 'y2': 0},   # East
            {'name': '27', 'heading': 270, 'x1': 2, 'y1': 0, 'x2': -2, 'y2': 0},  # West
            {'name': '04', 'heading': 45, 'x1': -1.4, 'y1': -1.4, 'x2': 1.4, 'y2': 1.4},  # NE
            {'name': '22', 'heading': 225, 'x1': 1.4, 'y1': 1.4, 'x2': -1.4, 'y2': -1.4}, # SW
        ]
        
        # State
        self.aircraft: List[Aircraft] = []
        self.time_elapsed = 0.0
        self.last_spawn_time = 0.0
        self.score = 0
        self.violations = 0
        self.conflicts = 0
        self.successful_landings = 0
        
        # Matplotlib figure for rendering
        self.fig = None
        self.ax = None
        
        # Define observation space
        # Per-aircraft: x, y, altitude, heading, speed, target_alt, target_hdg, target_spd,
        #               dx (to nearest runway), dy, runway_assigned, is_landing, distance, bearing
        self.observation_space = spaces.Dict({
            'aircraft': spaces.Box(
                low=-np.inf,
                high=np.inf,
                shape=(max_aircraft, 14),
                dtype=np.float32
            ),
            'aircraft_mask': spaces.Box(
                low=0,
                high=1,
                shape=(max_aircraft,),
                dtype=np.uint8
            ),
            'conflict_matrix': spaces.Box(
                low=0.0,
                high=1.0,
                shape=(max_aircraft, max_aircraft),
                dtype=np.float32
            ),
            'global_state': spaces.Box(
                low=-np.inf,
                high=np.inf,
                shape=(4,),
                dtype=np.float32
            )
        })
        
        # Define action space (Dict)
        # aircraft_id: which aircraft to command (0 to max_aircraft-1, or max_aircraft for no-op)
        # command_type: 0=altitude, 1=heading, 2=speed, 3=land, 4=no-op
        # altitude: 0-17 (0ft, 1000ft, 2000ft, ..., 17000ft)
        # heading: 0-11 (0, 30, 60, ..., 330 degrees)
        # speed: 0-7 (150, 180, 210, 240, 270, 300, 330, 360 knots)
        self.action_space = spaces.Dict({
            'aircraft_id': spaces.Discrete(max_aircraft + 1),
            'command_type': spaces.Discrete(5),
            'altitude': spaces.Discrete(18),
            'heading': spaces.Discrete(12),
            'speed': spaces.Discrete(8),
        })
    
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        
        self.aircraft = []
        self.time_elapsed = 0.0
        self.last_spawn_time = 0.0
        self.score = 0
        self.violations = 0
        self.conflicts = 0
        self.successful_landings = 0
        
        # Spawn initial aircraft
        for _ in range(min(3, self.max_aircraft)):
            self._spawn_aircraft()
        
        obs = self._get_observation()
        info = self._get_info()
        
        return obs, info
    
    def _spawn_aircraft(self):
        """Spawn a new aircraft at the edge of the simulation area."""
        if len(self.aircraft) >= self.max_aircraft:
            return
        
        # Random spawn position at edge
        edge = self.np_random.integers(0, 4)  # 0=North, 1=East, 2=South, 3=West
        
        if edge == 0:  # North
            x = self.np_random.uniform(-AREA_SIZE_NM/2, AREA_SIZE_NM/2)
            y = AREA_SIZE_NM / 2
            heading = 180  # South
        elif edge == 1:  # East
            x = AREA_SIZE_NM / 2
            y = self.np_random.uniform(-AREA_SIZE_NM/2, AREA_SIZE_NM/2)
            heading = 270  # West
        elif edge == 2:  # South
            x = self.np_random.uniform(-AREA_SIZE_NM/2, AREA_SIZE_NM/2)
            y = -AREA_SIZE_NM / 2
            heading = 0  # North
        else:  # West
            x = -AREA_SIZE_NM / 2
            y = self.np_random.uniform(-AREA_SIZE_NM/2, AREA_SIZE_NM/2)
            heading = 90  # East
        
        altitude = self.np_random.uniform(3000, 10000)
        speed = self.np_random.uniform(200, 280)
        
        callsign = CALLSIGNS[len(self.aircraft) % len(CALLSIGNS)]
        
        aircraft = Aircraft(
            callsign=callsign,
            x=x,
            y=y,
            altitude=altitude,
            heading=heading,
            speed=speed,
        )
        
        self.aircraft.append(aircraft)
    
    def step(self, action: Dict[str, int]):
        dt = 1.0  # 1 second per step
        
        # Execute action
        reward = self._execute_action(action)
        
        # Update all aircraft
        for aircraft in self.aircraft:
            aircraft.update(dt)
        
        # Check for conflicts/violations
        conflict_penalty, violation_penalty = self._check_conflicts()
        reward += conflict_penalty + violation_penalty
        
        # Remove aircraft that left the area
        self.aircraft = [
            ac for ac in self.aircraft
            if abs(ac.x) <= AREA_SIZE_NM/2 and abs(ac.y) <= AREA_SIZE_NM/2
        ]
        
        # Check for successful landings
        landing_reward = self._check_landings()
        reward += landing_reward
        
        # Spawn new aircraft periodically
        self.time_elapsed += dt
        if self.time_elapsed - self.last_spawn_time >= self.spawn_interval:
            self._spawn_aircraft()
            self.last_spawn_time = self.time_elapsed
        
        # Update score
        self.score += reward
        
        # Check termination
        terminated = self.time_elapsed >= self.episode_length
        truncated = False
        
        obs = self._get_observation()
        info = self._get_info()
        
        return obs, reward, terminated, truncated, info
    
    def _execute_action(self, action: Dict[str, int]) -> float:
        """Execute the commanded action and return immediate reward."""
        aircraft_id = action['aircraft_id']
        command_type = action['command_type']
        
        # No-op
        if aircraft_id >= len(self.aircraft) or command_type == 4:
            return 0.0
        
        aircraft = self.aircraft[aircraft_id]
        reward = 0.0
        
        if command_type == 0:  # Altitude
            altitude_ft = action['altitude'] * 1000
            aircraft.target_altitude = altitude_ft
            reward = 0.1  # Small reward for issuing command
        
        elif command_type == 1:  # Heading
            heading_deg = action['heading'] * 30
            aircraft.target_heading = heading_deg
            reward = 0.1
        
        elif command_type == 2:  # Speed
            speed_kts = 150 + action['speed'] * 30
            aircraft.target_speed = speed_kts
            reward = 0.1
        
        elif command_type == 3:  # Land
            # Assign to nearest runway
            best_runway = None
            best_dist = float('inf')
            for i, runway in enumerate(self.runways):
                dist = np.sqrt((aircraft.x - runway['x2'])**2 + (aircraft.y - runway['y2'])**2)
                if dist < best_dist:
                    best_dist = dist
                    best_runway = i
            
            aircraft.is_landing = True
            aircraft.runway_assigned = best_runway
            aircraft.target_altitude = 0
            aircraft.target_speed = 150
            reward = 0.5  # Reward for initiating landing
        
        return reward
    
    def _check_conflicts(self) -> Tuple[float, float]:
        """Check for conflicts and violations between aircraft."""
        conflict_penalty = 0.0
        violation_penalty = 0.0
        
        for i, ac1 in enumerate(self.aircraft):
            for j, ac2 in enumerate(self.aircraft[i+1:], start=i+1):
                is_violation, is_conflict = ac1.check_conflict(ac2)
                
                if is_violation:
                    violation_penalty -= 10.0
                    self.violations += 1
                elif is_conflict:
                    conflict_penalty -= 1.0
                    self.conflicts += 1
        
        return conflict_penalty, violation_penalty
    
    def _check_landings(self) -> float:
        """Check for successful landings and remove landed aircraft."""
        reward = 0.0
        aircraft_to_remove = []
        
        for ac in self.aircraft:
            if ac.is_landing and ac.altitude < 100:
                # Check if close to runway
                if ac.runway_assigned is not None:
                    runway = self.runways[ac.runway_assigned]
                    dist_to_runway = np.sqrt((ac.x - runway['x2'])**2 + (ac.y - runway['y2'])**2)
                    
                    if dist_to_runway < 0.5:  # Within 0.5nm of runway end
                        reward += 20.0
                        self.successful_landings += 1
                        aircraft_to_remove.append(ac)
        
        for ac in aircraft_to_remove:
            self.aircraft.remove(ac)
        
        return reward
    
    def _get_observation(self) -> Dict[str, np.ndarray]:
        """Get current observation."""
        aircraft_features = np.zeros((self.max_aircraft, 14), dtype=np.float32)
        aircraft_mask = np.zeros(self.max_aircraft, dtype=np.uint8)
        conflict_matrix = np.zeros((self.max_aircraft, self.max_aircraft), dtype=np.float32)
        
        for i, ac in enumerate(self.aircraft):
            if i >= self.max_aircraft:
                break
            
            # Calculate distance and bearing to airport
            distance = np.sqrt(ac.x**2 + ac.y**2)
            bearing = np.degrees(np.arctan2(ac.x, ac.y)) % 360
            
            # Find nearest runway
            nearest_runway_dx = 0
            nearest_runway_dy = 0
            if ac.runway_assigned is not None:
                runway = self.runways[ac.runway_assigned]
                nearest_runway_dx = runway['x2'] - ac.x
                nearest_runway_dy = runway['y2'] - ac.y
            
            aircraft_features[i] = [
                ac.x / AREA_SIZE_NM,
                ac.y / AREA_SIZE_NM,
                ac.altitude / 10000.0,
                ac.heading / 360.0,
                ac.speed / 300.0,
                ac.target_altitude / 10000.0,
                ac.target_heading / 360.0,
                ac.target_speed / 300.0,
                nearest_runway_dx / AREA_SIZE_NM,
                nearest_runway_dy / AREA_SIZE_NM,
                float(ac.runway_assigned if ac.runway_assigned is not None else -1) / 4.0,
                float(ac.is_landing),
                distance / AREA_SIZE_NM,
                bearing / 360.0,
            ]
            aircraft_mask[i] = 1
        
        # Compute conflict matrix
        for i, ac1 in enumerate(self.aircraft):
            for j, ac2 in enumerate(self.aircraft):
                if i != j and i < self.max_aircraft and j < self.max_aircraft:
                    is_violation, is_conflict = ac1.check_conflict(ac2)
                    if is_violation:
                        conflict_matrix[i, j] = 1.0
                    elif is_conflict:
                        conflict_matrix[i, j] = 0.5
        
        global_state = np.array([
            len(self.aircraft) / self.max_aircraft,
            self.time_elapsed / self.episode_length,
            self.score / 100.0,
            self.violations / 10.0,
        ], dtype=np.float32)
        
        return {
            'aircraft': aircraft_features,
            'aircraft_mask': aircraft_mask,
            'conflict_matrix': conflict_matrix,
            'global_state': global_state,
        }
    
    def _get_info(self) -> Dict[str, Any]:
        return {
            'time_elapsed': self.time_elapsed,
            'num_aircraft': len(self.aircraft),
            'score': self.score,
            'violations': self.violations,
            'conflicts': self.conflicts,
            'successful_landings': self.successful_landings,
        }
    
    def render(self):
        if self.render_mode != 'human':
            return
        
        if self.fig is None:
            self.fig, self.ax = plt.subplots(figsize=(10, 10))
        
        self.ax.clear()
        self.ax.set_xlim(-AREA_SIZE_NM/2, AREA_SIZE_NM/2)
        self.ax.set_ylim(-AREA_SIZE_NM/2, AREA_SIZE_NM/2)
        self.ax.set_aspect('equal')
        self.ax.set_facecolor('black')
        self.fig.patch.set_facecolor('black')
        
        # Draw runways (white lines)
        for runway in self.runways:
            self.ax.plot(
                [runway['x1'], runway['x2']],
                [runway['y1'], runway['y2']],
                color='white',
                linewidth=3,
                label=runway['name']
            )
        
        # Draw aircraft (yellow triangles)
        for ac in self.aircraft:
            # Triangle pointing in heading direction
            heading_rad = np.radians(ac.heading)
            
            # Triangle vertices (pointing up by default)
            size = 0.5
            vertices = np.array([
                [0, size],      # tip
                [-size/2, -size/2],  # left
                [size/2, -size/2],   # right
            ])
            
            # Rotate by heading
            cos_h = np.cos(heading_rad)
            sin_h = np.sin(heading_rad)
            rotation_matrix = np.array([
                [sin_h, cos_h],
                [cos_h, -sin_h]
            ])
            rotated = vertices @ rotation_matrix.T
            
            # Translate to aircraft position
            rotated[:, 0] += ac.x
            rotated[:, 1] += ac.y
            
            triangle = Polygon(
                rotated,
                closed=True,
                facecolor='yellow',
                edgecolor='orange',
                linewidth=1
            )
            self.ax.add_patch(triangle)
            
            # Label with callsign and altitude
            self.ax.text(
                ac.x, ac.y + 0.8,
                f"{ac.callsign}\n{int(ac.altitude)}ft",
                color='white',
                fontsize=8,
                ha='center',
                va='bottom'
            )
        
        # Add info text
        info_text = (
            f"Time: {self.time_elapsed:.1f}s\n"
            f"Aircraft: {len(self.aircraft)}\n"
            f"Score: {self.score:.1f}\n"
            f"Violations: {self.violations}\n"
            f"Landings: {self.successful_landings}"
        )
        self.ax.text(
            -AREA_SIZE_NM/2 + 1, AREA_SIZE_NM/2 - 1,
            info_text,
            color='white',
            fontsize=10,
            va='top',
            bbox=dict(boxstyle='round', facecolor='black', alpha=0.7)
        )
        
        self.ax.grid(True, color='gray', alpha=0.3)
        self.ax.set_xlabel('X (nautical miles)', color='white')
        self.ax.set_ylabel('Y (nautical miles)', color='white')
        self.ax.tick_params(colors='white')
        
        plt.tight_layout()
        plt.pause(0.01)
    
    def close(self):
        if self.fig is not None:
            plt.close(self.fig)
            self.fig = None
            self.ax = None

print('✅ RunwayEnvironment class defined')

NameError: name 'gym' is not defined

# Section 4: Test the Environment

In [ ]:
# Create environment
env = RunwayEnvironment(
    max_aircraft=10,
    episode_length=120,  # 2 minutes
    spawn_interval=20.0,
    render_mode='human'
)

print('✅ Environment created')
print(f'Observation space: {env.observation_space}')
print(f'Action space: {env.action_space}')

ValueError: Box low must be a np.ndarray, integer, or float, actual type=<class 'bool'>

: 

In [ ]:
# Reset and render
obs, info = env.reset()
env.render()

print(f"Initial state:")
print(f"  Aircraft: {info['num_aircraft']}")
print(f"  Time: {info['time_elapsed']:.1f}s")
print(f"  Score: {info['score']}")

In [ ]:
# Run a few random steps to see the simulation
print('Running simulation for 30 seconds...')

for step in range(30):
    # Random action
    action = env.action_space.sample()
    
    obs, reward, terminated, truncated, info = env.step(action)
    
    if step % 5 == 0:  # Render every 5 steps
        clear_output(wait=True)
        env.render()
        print(f"Step {step}: Aircraft={info['num_aircraft']}, Score={info['score']:.1f}, Violations={info['violations']}")
        time.sleep(0.2)
    
    if terminated or truncated:
        break

print(f"\nFinal state:")
print(f"  Aircraft: {info['num_aircraft']}")
print(f"  Time: {info['time_elapsed']:.1f}s")
print(f"  Score: {info['score']:.1f}")
print(f"  Violations: {info['violations']}")
print(f"  Landings: {info['successful_landings']}")

# Section 5: Train with PPO

In [ ]:
# Close previous environment
env.close()

# Create training environment (no rendering)
train_env = RunwayEnvironment(
    max_aircraft=8,
    episode_length=180,
    spawn_interval=25.0,
    render_mode=None  # No rendering during training
)

# Vectorize for SB3
vec_env = DummyVecEnv([lambda: train_env])

print('✅ Training environment created')

In [ ]:
# Create PPO model
model = PPO(
    "MultiInputPolicy",
    vec_env,
    learning_rate=3e-4,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    verbose=1,
)

print('✅ PPO model created')

In [ ]:
# Train the model (start with 10k steps for quick test)
print('Training for 10,000 steps...')
model.learn(total_timesteps=10_000, progress_bar=True)

print('\n✅ Training complete!')
print('For better results, train for 100k+ steps')

# Section 6: Evaluate Trained Model

In [ ]:
# Create evaluation environment with rendering
eval_env = RunwayEnvironment(
    max_aircraft=8,
    episode_length=120,
    spawn_interval=20.0,
    render_mode='human'
)

print('Evaluating trained model...')

obs, info = eval_env.reset()
eval_env.render()

for step in range(120):
    # Use trained model to predict action
    action, _states = model.predict(obs, deterministic=True)
    
    # Convert numpy array to dict action
    action_dict = {
        'aircraft_id': int(action['aircraft_id'][0]),
        'command_type': int(action['command_type'][0]),
        'altitude': int(action['altitude'][0]),
        'heading': int(action['heading'][0]),
        'speed': int(action['speed'][0]),
    }
    
    obs, reward, terminated, truncated, info = eval_env.step(action_dict)
    
    if step % 5 == 0:
        clear_output(wait=True)
        eval_env.render()
        print(f"Step {step}: Score={info['score']:.1f}, Violations={info['violations']}, Landings={info['successful_landings']}")
        time.sleep(0.2)
    
    if terminated or truncated:
        break

print(f"\nEvaluation Results:")
print(f"  Final Score: {info['score']:.1f}")
print(f"  Violations: {info['violations']}")
print(f"  Successful Landings: {info['successful_landings']}")

eval_env.close()

# Section 7: Summary

## What We Built

✅ **Self-Contained Environment** - No browser, pure Python simulation  
✅ **Visual Rendering** - Black canvas, white runways, yellow aircraft triangles  
✅ **Realistic Physics** - Based on OpenScope dynamics (turn rates, climb/descent)  
✅ **Safety Rules** - 3nm lateral, 1000ft vertical separation  
✅ **RL Training** - Full Gymnasium + Stable-Baselines3 PPO integration  

## Key Components

1. **Aircraft Class**: Position, heading, altitude, speed with realistic physics
2. **RunwayEnvironment**: Gymnasium environment with:
   - Two intersecting runways (4 landing directions)
   - Aircraft spawning from edges
   - Conflict/violation detection
   - Landing detection
   - Visual rendering with matplotlib
3. **Action Space**: Aircraft selection + command (altitude/heading/speed/land)
4. **Observation Space**: Per-aircraft features, conflict matrix, global state

## Performance

- **Much faster** than browser-based simulation
- Can train with multiple parallel environments
- No Playwright/browser dependencies

## Next Steps

**Improve Training**:
- Train for 100k+ steps for better results
- Add curriculum learning (start with fewer aircraft)
- Tune reward function (landing bonuses, efficiency rewards)

**Extend Environment**:
- Add wind effects
- More complex runway configurations
- Different aircraft types (speed ranges)
- Emergencies and go-arounds

**Advanced RL**:
- Use custom Transformer policy (see main project)
- Try other algorithms (SAC, TD3)
- Multi-agent coordination

**Resources**: See `CLAUDE.md` for main project documentation